# Inferring signaling networks from multiple interventions with CORNETO

This notebook illustrates how CORNETO can be used to learn the intracellular signaling network of a cell from multiple drug perturbations using prior knowledge, for which drug targets and RNA-seq data per perturbation is available. For this, we use the data from the [PANACEA DREAM Challenge](https://doi.org/10.1016/j.xcrm.2021.100492), which contains the gene expression profiles of cancer cell-lines treated with 32 different kinase inhibitors with known targets. The pipeline starts from two data files:

- The gene counts matrix for the DU145 cell line, which can be downloaded from [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE186341)
- The drug-target information, which can be obtained from the supplementary materials of the paper [here](https://ars.els-cdn.com/content/image/1-s2.0-S2666379121003694-mmc3.xlsx).

The method infers the intracellular signaling network that recapitulates the observed changes in TF activities for each drug, by finding the minimal sub-graph from a Prior Knowledge Network that captures the signaling changes from drug targets to TFs. Typically, in a single contrast setting, the objective is to minimize the error between the predicted and observed changes in TF activity, with a regularization term that controls the size of the solution. This is how [CARNIVAL method](https://www.nature.com/articles/s41540-019-0118-z) works. In contrast, in the multi-condition setting, we extend this principle by minimizing the error between the predicted and observed changes in node activity across multiple conditions, while still controlling the size of the final solution in a single optimization problem. This results in a network that identifies better the signaling cascades by exploiting multiple independent interventions at the same time, instead of doing it independently.


## Data preprocessing

To run the analysis, we need to prepare the following data:

- A signaling prior knowledge network, in the form of an activity flow network, which we obtain from [OmniPath](https://omnipathdb.org/).
- The perturbed nodes in the network, which we obtain from the drug-target information.
- The TF activity changes, which we obtain from the differential gene expression profiles using [DoRoThea](https://saezlab.github.io/dorothea/) and DecoupleR (https://saezlab.github.io/decoupleR).

Processing of PANACEA data is done with processing.R script.



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# libraries, read data and prepare reaction network
import numpy as np
import pandas as pd
import corneto as cnt
from corneto import Graph, create_flow_graph, signflow
import os
import re

print(cnt.__version__)

In [ ]:
# Get a list of all the folders in the current directory that start with 'run_'
folders = [f for f in os.listdir('output/') if f.startswith('run_')]
# Extract the numbers from the folders
numbers = [int(re.search(r'\d+', f).group()) for f in folders]
# If there are no folders that start with 'run_', start the numbering at 1
if not numbers:
    next_number = 1
else:
    # Otherwise, find the next number by adding 1 to the maximum number in the list
    next_number = max(numbers) + 1
# Create the new folder with the next number
folder = f"output/run_{next_number:04d}"
os.mkdir(folder)
print(f"Created folder {folder}")

In [ ]:
# load data
panacea_df = pd.read_csv("data.tsv", sep="\t")
pkn_df = pd.read_csv("pkn.tsv", sep="\t")

# filter panacea df only to features taht are in the source or target column of the pkn
panacea_df = panacea_df[panacea_df["feature"].isin(pkn_df['source'].unique()) | panacea_df["feature"].isin(pkn_df['target'].unique())]

pkn = cnt.import_sif("pkn.tsv", has_header=True)

# get perturbation_list as the list of unique ids in the feature column of the panacea df where the type is "perturbation"
perturbation_list = panacea_df[panacea_df["type"] == "perturbation"]["feature"].unique().tolist()
measurement_list = panacea_df[panacea_df["type"] == "measurement"]["feature"].unique().tolist()

# prune the reaction network and prepare it for downstream analysis
pruned_pkn = pkn.prune(perturbation_list, measurement_list)
network = Graph.import_network(pruned_pkn)
print(len(pkn.species), len(pkn.reactions))
print(len(pruned_pkn.species), len(pruned_pkn.reactions))

In [ ]:
# prepare corneto input
input_data = {}

# iterate over each compound
compounds = panacea_df["compound"].unique()

#### REMOVE THIS TO EXTEND TO ALL THE PERTURBATIONS ####
#compounds = compounds[1:3]
########################################################

# iterate over compounds
for compound in compounds:
    compound_df = panacea_df[panacea_df["compound"] == compound]
    pert_df = compound_df[compound_df["type"] == "perturbation"]
    meas_df = compound_df[compound_df["type"] == "measurement"]
    # create dictionaries
    pert_dict = {}
    meas_dict = {}
    # iterate over pert_df rows
    for index, row in pert_df.iterrows():
        pert_dict[row["feature"]] = ("P", row["score"])
    for index, row in meas_df.iterrows():
        meas_dict[row["feature"]] = ("M", row["score"])
    
    # concatenate both dictionaries
    input_data[compound] = {**pert_dict, **meas_dict}

flow_graph = create_flow_graph(network, input_data)
print(flow_graph.num_species, flow_graph.num_reactions)

In [ ]:
# Print conditions
input_data.keys()

In [ ]:
cp = signflow(
    flow_graph, 
    input_data, 
    l0_penalty_reaction = 0.1, 
    l0_penalty_species = 0.1
)

P = cp.solve(
    'GUROBI', 
    verbosity=1,
    NoRelHeurTime=8000,
    MIPGap=0.02,
    Method=1,
    TimeLimit=10000
)

In [ ]:
for o in cp.objectives:
    print(o.value)

In [ ]:
# Defined variables in the flow problem
cp.symbols

In [ ]:
def get_sol(pb, net):
    df_edges = pd.DataFrame(
        {'edges': net.reactions, 
         'flow': pb.symbols['_flow_rxn'].value,
         'flow_indicator': np.abs(pb.symbols['_flow_rxn_ipos'].value)
        }).set_index('edges')
    for c in compounds:
        act = np.abs(pb.symbols[f'reaction_sends_activation_{c}'].value)
        inh = np.abs(pb.symbols[f'reaction_sends_inhibition_{c}'].value)
        val = act - inh
        df_edges[f'edge_value_{c}'] = val
    df_nodes = pd.DataFrame({'nodes': net.species}).set_index('nodes')
    for c in compounds:
        act = np.abs(pb.symbols[f'species_activated_{c}'].value)
        inh = np.abs(pb.symbols[f'species_inhibited_{c}'].value)
        val = act - inh
        df_nodes[f'{c}'] = val
    return df_nodes, df_edges

df_nodes, df_edges = get_sol(cp, flow_graph)
df_nodes

In [ ]:
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors

normalize = mcolors.TwoSlopeNorm(vcenter=0, vmin=-1, vmax=1)
sns.clustermap(df_nodes.loc[df_nodes.abs().sum(axis=1) > 1,:], cmap=cm.RdBu_r, norm=normalize)
plt.savefig(f'{folder}/heatmap_activity_nodes.pdf', format='pdf')

In [ ]:
# Remove dummy nodes, drug targets and TFs and compare only unoobserved nodes
observed = set()
for k, v in input_data.items():
    observed |= set(v.keys())
predicted_nodes = list(set(df_nodes.index[~df_nodes.index.str.startswith('_')].values.tolist()).difference(observed))

df_nodes_pred = df_nodes.loc[predicted_nodes,:]
sns.clustermap(df_nodes_pred.loc[df_nodes_pred.abs().sum(axis=1) > 1,:], cmap=cm.RdBu_r, norm=normalize)

In [ ]:
sns.clustermap(df_nodes.loc[df_nodes.abs().sum(axis=1) > 1,:].corr(), cmap=cm.RdBu_r, norm=normalize)
plt.savefig(f'{folder}/heatmap_activity_nodes_corr_drugs.pdf', format='pdf')

In [ ]:
sns.clustermap(df_nodes_pred.loc[df_nodes_pred.abs().sum(axis=1) > 1,:].corr(), cmap=cm.RdBu_r, norm=normalize)
plt.savefig(f'{folder}/heatmap_activity_nodes_corr_drugs_only_predicted.pdf', format='pdf')

In [ ]:
df_edges.to_csv(f'{folder}/sol_edges.csv')
df_nodes.to_csv(f'{folder}/sol_nodes.csv')
df_nodes.loc[df_nodes.abs().sum(axis=1) > 1,:].corr().to_csv(f'{folder}/node_values_corr.csv')
df_nodes_pred.loc[df_nodes_pred.abs().sum(axis=1) > 1,:].corr().to_csv(f'{folder}/node_values_corr_only_predicted.csv')

In [ ]:
df_nodes.std(axis=1).sort_values(ascending=False).head(30)

In [ ]:
pd.DataFrame(df_nodes.abs().sum(axis=1).sort_values(ascending=False)).to_csv(f'{folder}/node_counts_sol.csv')
pd.DataFrame(df_nodes.std(axis=1).sort_values(ascending=False)).to_csv(f'{folder}/node_std_sol.csv')